In [1]:
!pip install geopy

     |████████████████████████████████| 118 kB 5.5 MB/s eta 0:00:01


In [1]:
from geopy.distance import geodesic as GD

In [2]:
import numpy as np
import pandas as pd
import csv
from collections import namedtuple
from collections import defaultdict
import math
import time

In [9]:
df = pd.read_csv('nyc_taxi_data.csv')

df['pickup_longitude'] = df['pickup_longitude'].round(4)

df['pickup_latitude'] = df['pickup_latitude'].round(4)

df['dropoff_longitude'] = df['dropoff_longitude'].round(4)

df['dropoff_latitude'] = df['dropoff_latitude'].round(4)

In [10]:
df = df[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','trip_distance']]

In [11]:
weighted_edges = df.values.tolist()
from geopy.distance import geodesic as GD

heu =[]

for i in weighted_edges:
    nyc1 =(i[0],i[1])
    nyc2 = (i[2],i[3])
    heuristic_val = GD(nyc1,nyc2).km
    i.append(heuristic_val)
    heu.append(heuristic_val)
df['heu'] = heu

In [ ]:
df['from_route'] = df['pickup_longitude'].astype(str) +"-"+ df["pickup_latitude"].astype(str)
df['to_route'] = df['dropoff_longitude'].astype(str) +"-"+ df["dropoff_latitude"].astype(str)

node_dict = {}

for i in range (len(df)):
    if df['from_route'][i] in node_dict:
        pass
    else:
        node_dict[df['from_route'][i]] = i 
        
    if df['to_route'][i] in node_dict:
        pass
    else:
        node_dict[df['to_route'][i]] = i+ 168152

In [ ]:
new_df = pd.DataFrame()
new_df['node'] = node_dict.values()

ll = pd.Series(node_dict.keys())
new_df['lat_long'] = node_dict.keys()

df['node_pickup'] = df['from_route'].map(new_df.set_index('lat_long')['node'])
df['node_dropoff'] = df['to_route'].map(new_df.set_index('lat_long')['node'])

In [12]:
graph = defaultdict(list)
for i in weighted_edges:
    pick_cord=(i[0],i[1])
    graph[pick_cord].append([ i[2],i[3],i[4],i[5]])


In [15]:
a  = list(df[['node_pickup', 'node_dropoff', 'trip_distance', 'heu']].values)

from collections import defaultdict

def generateDirectedGraph(edges):
    graph = defaultdict(dict)
    for u, v, dist, heu in edges:
        graph[u][v] = dist, heu
        graph[v][u] = dist, heu
    return graph
  
weighted_edges = a
d_wt_grph = generateDirectedGraph(weighted_edges)

defaultdict(dict,
            {1: {2: (2.4, 1.6894855129839041),
              3: (1.7, 3.1129786310005434),
              4: (0.82, 1.023677012291488),
              5: (3.1, 3.7039722533204325),
              6: (1.0, 0.47667309932425267)},
             2: {1: (2.4, 1.6894855129839041),
              76693: (1.28, 1.5990345357845408),
              14661: (2.08, 2.4282294077929403),
              24996: (1.63, 1.630724870942278)},
             3: {1: (1.7, 3.1129786310005434)},
             4: {1: (0.82, 1.023677012291488),
              54894: (1.89, 2.4073265644096646),
              26733: (1.19, 2.020143493962763),
              159: (8.91, 11.232414704732738)},
             5: {1: (3.1, 3.7039722533204325),
              56536: (1.09, 0.4133553837333741),
              59220: (4.18, 1.4593202662391944),
              59221: (2.3, 1.3931543454829078),
              59222: (5.6, 2.973534612065138),
              59223: (2.0, 0.995766518833487),
              59224: (2.37, 2.076228

In [16]:
start = time.time()
def heu_cal(cell, goal):
    source_cord = (list(lat_long_id.keys())[list(lat_long_id.values()).index(cell)])
    dest_coor = (list(lat_long_id.keys())[list(lat_long_id.values()).index(goal)])
    heu_val = GD(source_cord,dest_coor).km
    return heu_val
    

def r_constr_pth(pat, curr):
    final_path = [curr]
    while curr in pat:
        curr = pat[curr]
        final_path.append(curr)
    return final_path


def a_(graph, start, goal):

    c_s = []
    o_s = [start]
    pat ={}
    gscore ={}
    for key in graph:
        gscore[key] = 10000
    gscore[start] = 0
    fscore = {}
    for key in graph:
        fscore[key] = 1000000
    fscore[start] = heu_cal(start, goal)
    
    
    while o_s:
        min_val = 10000
        for node in o_s:
            if fscore[node] < min_val:
                min_val = fscore[node]
                min_node = node
        curr = min_node
        
        if curr == goal:
            return r_constr_pth(pat, curr)
        o_s.remove(curr)
        c_s.append(curr)
        
        for neigh in graph[curr]:
            if neigh in c_s:
                continue
            if neigh not in o_s:
                o_s.append(neigh)
            tentative_gscore = gscore[curr] + 1
            
            if tentative_gscore >= gscore[neigh]:
                continue
            pat[neigh] = curr  # record the best path untill now
            gscore[neigh] = tentative_gscore
            fscore[neigh] = gscore[neigh] + heu_cal(start, goal)
    print("here")
    return False

In [17]:
end = time.time()
shortest_route = a_(directed_weighted_graph, 1, 10)
time = end - start

In [18]:
dist = 0
for i in range(len(shortest_route) - 1):
    key =shortest_route[i]
    j = i +1
    key2 = shortest_route[j]
    dist = dist + directed_weighted_graph[key][key2][0]
    
print('path is:{shortest_route}')   
print('path distance is :{dist}')    
print('time take is:{time}')

path is:[10, 19717, 1936, 1928, 1682, 26733, 4, 1]
path distance is :33.419999999999995
time take is:0.0037109851837158203
